In [ ]:
# 📦 Importações
import pandas as pd
import numpy as np
from collections import defaultdict
import warnings
import plotly.express as px
from IPython.display import display

# 🔽 Carregar JSON e normalizar colunas aninhadas
url = 'https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/refs/heads/main/TelecomX_Data.json'
df = pd.read_json(url)

df = pd.concat([
    df.drop(['customer', 'phone', 'internet', 'account'], axis=1),
    pd.json_normalize(df['customer']),
    pd.json_normalize(df['phone']),
    pd.json_normalize(df['internet']),
    pd.json_normalize(df['account'])
], axis=1)

# 🔁 Converter valores categóricos
# "yes" → 1 ; "no", "no phone service", "no internet service" → 0
yes_val = "yes"
zero_vals = ["no", "no phone service", "no internet service"]

for col in df.select_dtypes(include=["object", "string"]).columns:
    df[col] = df[col].map(
        lambda x: 1 if isinstance(x, str) and x.strip().lower() == yes_val
        else 0 if isinstance(x, str) and x.strip().lower() in zero_vals
        else x
    )

# Substituir strings vazias por NaN (evita problema em colunas com espaços)
pd.set_option('future.no_silent_downcasting', True)

cols_obj = df.select_dtypes(include=["object", "string"]).columns
df[cols_obj] = df[cols_obj].replace("", np.nan).infer_objects(copy=False)


# Converter valores monetários (que podem ter vindo como string) para float
df["Charges.Monthly"] = pd.to_numeric(df["Charges.Monthly"], errors="coerce")
df["Charges.Total"]   = pd.to_numeric(df["Charges.Total"], errors="coerce")

# Converter a coluna Churn para inteiro (0 ou 1)
# Opção A: Remover linhas com Churn ausente
# 🔄 Remover valores ausentes em Churn e evitar o warning
df = df[df["Churn"].notna()].copy()

# ✅ Agora pode converter para int com segurança
df["Churn"] = df["Churn"].astype("int64")

df = df[df["Charges.Total"].notna()].copy()



# 🧮 Criar a coluna 'Contas_Diarias' com base no faturamento mensal e arredondar para 2 casas decimais
df["Contas_Diarias"] = (df["Charges.Monthly"] / 30).round(2)


# 💾 (Opcional) Exportar para Excel
# df.to_excel("dados_limpos.xlsx", index=False)

# 👁️ Visualização rápida
display(df.head())


,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total,Contas_Diarias
0,0002-ORFBO,0,Female,0,1,1,9,1,0,DSL,...,0,1,1,0,One year,1,Mailed check,65.6,593.30,2.19
1,0003-MKNFE,0,Male,0,0,0,9,1,1,DSL,...,0,0,0,1,Month-to-month,0,Mailed check,59.9,542.40,2.00
2,0004-TLHLJ,1,Male,0,0,0,4,1,0,Fiber optic,...,1,0,0,0,Month-to-month,1,Electronic check,73.9,280.85,2.46
3,0011-IGKFF,1,Male,1,1,0,13,1,0,Fiber optic,...,1,0,1,1,Month-to-month,1,Electronic check,98.0,1237.85,3.27
4,0013-EXCHZ,1,Female,1,1,0,3,1,0,Fiber optic,...,0,1,1,0,Month-to-month,1,Mailed check,83.9,267.40,2.80


In [ ]:
warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    message=r"DataFrame\.applymap has been deprecated",
)

def safe_duplicate_count(df):
    mapper = getattr(df, "map", df.applymap)
    return int(
        mapper(lambda x: json.dumps(x, sort_keys=True)
               if isinstance(x, (dict, list, set)) else x)
        .duplicated().sum()
    )

def data_quality_report(df, sample_size=5):
    report = defaultdict(dict)

    # 1. Ausentes (NaN/None) -------------------------------------------------
    na_cols = df.isna().sum().loc[lambda s: s > 0]
    report["Valores ausentes (NA)"] = na_cols.to_dict()

    # 1‑b. Vazios '' ---------------------------------------------------------
    empty_cols = (df == '').sum().loc[lambda s: s > 0]
    report["Strings vazias ('')"] = empty_cols.to_dict()

    # 2. Duplicados ----------------------------------------------------------
    dup_rows = safe_duplicate_count(df)
    report["Duplicados"] = dup_rows

    # 3. Strings em colunas numéricas ---------------------------------------
    bad_numeric = {}
    for col in df.select_dtypes(include="object"):
        if df[col].dropna().map(type).isin([str, int, float, bool]).all():
            coerced = pd.to_numeric(
                df[col].astype(str).str.replace(",", ".", regex=False),
                errors="coerce"
            )
            non_num = coerced.isna() & df[col].notna()
            if non_num.any():
                bad_numeric[col] = int(non_num.sum())
    report["Strings em numéricas"] = bad_numeric

    # 4. Inconsistências em categorias --------------------------------------
    cat_issues = {}
    for col in df.select_dtypes(include="object"):
        if df[col].dropna().map(lambda x: isinstance(x, (dict, list))).any():
            continue
        orig = df[col].dropna().astype(str)
        norm = (orig.str.strip()
                     .str.lower()
                     .str.normalize("NFKD")
                     .str.replace(r"\s+", " ", regex=True))
        if len(orig.unique()) != len(norm.unique()):
            cat_issues[col] = norm.value_counts().head(sample_size).index.tolist()
    report["Categorias inconsistentes"] = cat_issues

    # 5. Colunas complexas ---------------------------------------------------
    complex_cols = [
        c for c in df.columns
        if df[c].dropna().map(lambda x: isinstance(x, (dict, list))).any()
    ]
    report["Colunas complexas"] = complex_cols

    # ---------------- PRINT -------------------------------------------------
    print("📋  RELATÓRIO DE QUALIDADE DOS DADOS\n")

    # 1. Ausentes
    print("1️⃣  Valores ausentes (NaN/None):")
    if na_cols.empty:
        print("   ✅ Nenhum NA\n")
    else:
        display(na_cols.to_frame("n_NA"))

    # 1‑b. Vazios
    print("1️⃣Strings vazias (''):\n")
    if empty_cols.empty:
        print("   ✅ Nenhum string vazia")
    else:
        display(empty_cols.to_frame("n_vazios"))

    # 2. Duplicados
    print("\n2️⃣  Linhas duplicadas:")
    print(f"   {'✅ Nenhuma duplicada' if dup_rows == 0 else f'⚠️  {dup_rows} duplicadas'}\n")

    # 3. Strings em numéricas
    print("3️⃣  Strings em colunas numéricas:")
    if not bad_numeric:
        print("   ✅ Nenhum problema\n")
    else:
        for col, q in bad_numeric.items():
            print(f"   ⚠️  {col}: {q} valores não numéricos")
        print()

    # 4. Categorias
    print("4️⃣  Inconsistências em categorias:")
    if not cat_issues:
        print("   ✅ Categorias OK\n")
    else:
        for col, ex in cat_issues.items():
            print(f"   ⚠️  {col}: exemplos → {ex}")
        print()

    # 5. Complexas
    print("5️⃣  Colunas com objetos complexos (dict/list):")
    if not complex_cols:
        print("   ✅ Nenhuma\n")
    else:
        print(f"   ⚠️  {complex_cols}\n"
              "   ➡️  Dica: normalize com `pd.json_normalize()` ou `explode()` antes do EDA.\n")

    print(df.info())

    return report


In [ ]:
relatorio = data_quality_report(df)

📋  RELATÓRIO DE QUALIDADE DOS DADOS

1️⃣  Valores ausentes (NaN/None):
   ✅ Nenhum NA

1️⃣Strings vazias (''):

   ✅ Nenhum string vazia

2️⃣  Linhas duplicadas:
   ✅ Nenhuma duplicada

3️⃣  Strings em colunas numéricas:
   ⚠️  customerID: 7032 valores não numéricos
   ⚠️  gender: 7032 valores não numéricos
   ⚠️  InternetService: 5512 valores não numéricos
   ⚠️  Contract: 7032 valores não numéricos
   ⚠️  PaymentMethod: 7032 valores não numéricos

4️⃣  Inconsistências em categorias:
   ✅ Categorias OK

5️⃣  Colunas com objetos complexos (dict/list):
   ✅ Nenhuma

<class 'pandas.core.frame.DataFrame'>
Index: 7032 entries, 0 to 7266
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7032 non-null   object 
 1   Churn             7032 non-null   int64  
 2   gender            7032 non-null   object 
 3   SeniorCitizen     7032 non-null   int64  
 4   Partner           7032 non-null   int64

In [ ]:
# Seleciona apenas colunas numéricas
df_numericas = df.select_dtypes(include=["int64", "float64"])

# Estatísticas descritivas principais
estatisticas_gerais = df_numericas.describe().T

# Calcula mediana separadamente
estatisticas_gerais["mediana"] = df_numericas.median()

# Inclui medidas adicionais
estatisticas_gerais["assimetria"] = df_numericas.skew()
estatisticas_gerais["curtose"] = df_numericas.kurt()

# Reorganiza colunas
estatisticas_gerais = estatisticas_gerais[["count", "mean", "mediana", "std", "min", "25%", "50%", "75%", "max", "assimetria", "curtose"]]

# Exibe
print("📊 Estatísticas descritivas das variáveis numéricas:")
display(estatisticas_gerais.round(2))


📊 Estatísticas descritivas das variáveis numéricas:


,count,mean,mediana,std,min,25%,50%,75%,max,assimetria,curtose
Churn,7032.0,0.27,0.00,0.44,0.00,0.00,0.00,1.00,1.00,1.06,-0.88
SeniorCitizen,7032.0,0.16,0.00,0.37,0.00,0.00,0.00,0.00,1.00,1.83,1.35
Partner,7032.0,0.48,0.00,0.50,0.00,0.00,0.00,1.00,1.00,0.07,-2.00
Dependents,7032.0,0.30,0.00,0.46,0.00,0.00,0.00,1.00,1.00,0.88,-1.22
tenure,7032.0,32.42,29.00,24.55,1.00,9.00,29.00,55.00,72.00,0.24,-1.39
PhoneService,7032.0,0.90,1.00,0.30,0.00,1.00,1.00,1.00,1.00,-2.73,5.45
MultipleLines,7032.0,0.42,0.00,0.49,0.00,0.00,0.00,1.00,1.00,0.32,-1.90
OnlineSecurity,7032.0,0.29,0.00,0.45,0.00,0.00,0.00,1.00,1.00,0.94,-1.11
OnlineBackup,7032.0,0.34,0.00,0.48,0.00,0.00,0.00,1.00,1.00,0.65,-1.57
DeviceProtection,7032.0,0.34,0.00,0.48,0.00,0.00,0.00,1.00,1.00,0.66,-1.57


In [ ]:


# Contagem e proporção
contagem_churn = df["Churn"].value_counts().sort_index()
proporcao_churn = df["Churn"].value_counts(normalize=True).sort_index() * 100

# Rótulos amigáveis
rotulos = {0: "Permaneceu", 1: "Cancelou"}

# Gráfico de pizza (proporção)
fig_pizza = px.pie(
    names=df["Churn"].map(rotulos),
    title="Distribuição dos Clientes por Churn",
    hole=0.4
)
fig_pizza.show()

# Gráfico de barras (quantidade)
df_churn_plot = contagem_churn.rename(index=rotulos).reset_index()
df_churn_plot.columns = ["Status", "Quantidade"]

fig_barras = px.bar(
    df_churn_plot,
    x="Status", y="Quantidade",
    title="Contagem de Clientes por Status de Churn",
    text_auto=True
)
fig_barras.show()



In [ ]:
category_orders={col: sorted(df[col].dropna().astype(str).unique())}



cat_cols = ['gender', 'Contract', 'PaymentMethod', 'InternetService']

for col in cat_cols:
    fig = px.histogram(
        df,
        x=col,
        color="Churn",
        barmode="group",  # use "relative" para proporção
        histnorm=None,
        title=f"Evasão por {col}",
        color_discrete_map={0: "green", 1: "red"},
        category_orders={col: sorted(df[col].dropna().astype(str).unique())}
    )
    fig.update_layout(
        xaxis_title=col,
        yaxis_title="Número de clientes",
        legend_title="Churn",
    )
    fig.show()



In [ ]:
# Seleciona apenas variáveis numéricas (já tratadas)
df_numericas = df.select_dtypes(include=["int64", "float64"])

# Matriz de correlação
matriz_corr = df_numericas.corr()

# Correlação com churn
corr_churn = matriz_corr["Churn"].sort_values(ascending=False)

print("📊 Correlação com a variável Churn:")
display(corr_churn.round(2))

# Existe correlação entre Contas_Diarias e Churn?

fig = px.scatter(
    df,
    x="Contas_Diarias",
    y="Churn",
    title="Relação entre Contas Diárias e Churn",
    trendline="ols",  # linha de tendência
    opacity=0.3
)
fig.show()

#Como a quantidade de serviços contratados impacta o churn?

# Cria coluna com a contagem de serviços ativos (considerando colunas que são 0 ou 1)
servico_cols = ['PhoneService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                'TechSupport', 'StreamingTV', 'StreamingMovies', 'MultipleLines']

df["Total_Servicos"] = df[servico_cols].sum(axis=1)

# Correlação com churn
print("🔁 Correlação entre número de serviços e churn:")
print(df[["Total_Servicos", "Churn"]].corr().round(2))

# Visualização
fig = px.box(
    df,
    x="Churn",
    y="Total_Servicos",
    title="Distribuição da Quantidade de Serviços por Status de Churn",
    labels={"Churn": "Status (0 = Permaneceu, 1 = Cancelou)", "Total_Servicos": "Total de Serviços"}
)
fig.show()


📊 Correlação com a variável Churn:


,Churn
Churn,1.00
Contas_Diarias,0.19
Charges.Monthly,0.19
PaperlessBilling,0.19
SeniorCitizen,0.15
StreamingTV,0.06
StreamingMovies,0.06
MultipleLines,0.04
PhoneService,0.01
DeviceProtection,-0.07


🔁 Correlação entre número de serviços e churn:
                Total_Servicos  Churn
Total_Servicos            1.00  -0.07
Churn                    -0.07   1.00


# 📊 Relatório Final — Análise de Evasão de Clientes (Churn)

---

## 🧭 Introdução

O objetivo deste projeto foi analisar a evasão de clientes (churn) em uma empresa de telecomunicações. A evasão representa os clientes que cancelam seus contratos, impactando diretamente a receita da empresa. Por meio da análise exploratória de dados, buscamos entender quais perfis de clientes têm maior propensão ao cancelamento, identificar padrões e gerar recomendações estratégicas baseadas em dados.

---

## 🧹 Limpeza e Tratamento de Dados

- 📥 Importação de dados em JSON com estrutura aninhada, utilizando `pd.json_normalize` para formatar os dados em colunas planas.
- 🔁 Conversão de variáveis categóricas como "Yes"/"No" para 1 e 0, incluindo variações como "No internet service".
- 🧼 Substituição de valores vazios por `NaN` e tratamento de colunas críticas como `Churn` e `Charges.Total`.
- 💰 Conversão das colunas `Charges.Monthly` e `Charges.Total` para `float`.
- ➗ Criação da variável `Contas_Diarias`, derivada de `Charges.Monthly / 30`.
- ➕ Criação da variável `Total_Servicos`, que contabiliza quantos serviços o cliente utiliza.

---

## 📊 Análise Exploratória de Dados

- Visualização da **distribuição de churn** com gráficos de pizza e barras.
- Análise de churn por variáveis categóricas: `gender`, `Contract`, `PaymentMethod` e `InternetService`.
- Estatísticas descritivas para todas as colunas numéricas, incluindo média, mediana, desvio padrão, assimetria e curtose.
- Análise de **correlação** entre variáveis numéricas, especialmente entre `Churn`, `Contas_Diarias` e `Total_Servicos`.
- Gráficos de dispersão e boxplots para visualizar a relação entre valor gasto e probabilidade de churn.

---

## 🧠 Conclusões e Insights

- Clientes com **contrato "Month-to-month"** apresentam maior propensão ao churn.
- Métodos de pagamento como **"Electronic check"** estão fortemente associados à evasão.
- Clientes que utilizam **menos serviços contratados** apresentam maior chance de cancelamento.
- A variável `Contas_Diarias` mostrou leve correlação com churn, sugerindo sensibilidade ao valor pago.

---

## ✅ Recomendações

1. Criar **incentivos de fidelidade** para clientes com contrato mensal.
2. **Melhorar a experiência** de clientes que usam métodos de pagamento de maior risco, como `Electronic check`.
3. Desenvolver **pacotes promocionais** que incentivem a contratação de múltiplos serviços.
4. Usar esses insights como base para **modelos de previsão de churn** com foco em ações preventivas.
5. Acompanhar continuamente os perfis de churn e ajustar estratégias com base nos dados mais recentes.

---

Este relatório oferece um panorama completo para auxiliar a empresa a **reduzir a evasão de clientes** e otimizar ações de retenção com base em dados reais.
""")


